In [203]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as fm

path = 'C:/Windows/Fonts/malgun.ttf'
font_name = fm.FontProperties(fname=path, size=50).get_name()
plt.rc('font', family=font_name)

In [204]:
import pandas as pd 
from sklearn.decomposition import TruncatedSVD
import numpy as np 
from sklearn.metrics.pairwise import cosine_similarity 

# 10개월(21년2월~22년 2월) 의 판매데이터 활용 
import os
path = './dataset/220317'
files = os.listdir(path)
files = [path+'/'+file for file in files][:-2]

df = pd.concat([pd.read_csv(c,encoding= "EUC-KR") for c in files[:10]])
df1 = pd.concat([pd.read_csv(c,encoding= "EUC-KR") for c in files[-2:]])
df = pd.concat([df,df1])

#전처리
# df.ITEM_NM = df.ITEM_NM.apply(lambda x: x.strip()).apply(lambda x: x.replace('\u3000','')).apply(lambda x: x.replace('(사용안함)',''))
df.ITEM_NM = df.ITEM_NM.apply(lambda x: x.strip()).apply(lambda x: x.replace('\u3000',''))
df.ITEM_NM.unique()

# 컬럼명 수정
columns = ['CMP_CD',
 'SALE_DT',
 'ITEM_PLU_CD',
 'ITEM_NM',
 'CUST_CD',
 'PAY_GNDR_DIV',
 'PAY_AGE',
 'SALE_QTY']
df.columns = columns
df['SALE_YM'] = df['SALE_DT'].apply(lambda x : str(x)[:6])
# 고객번호 별 판매데이터 생성 
df

,CMP_CD,SALE_DT,ITEM_PLU_CD,ITEM_NM,CUST_CD,PAY_GNDR_DIV,PAY_AGE,SALE_QTY,SALE_YM
0,BRKR,20210228,A52013,사랑에 빠진 딸기,1052909263NNH,2,24,1,202102
1,BRKR,20210228,A52013,사랑에 빠진 딸기,1038703175NNH,1,59,1,202102
2,BRKR,20210228,A52013,사랑에 빠진 딸기,1002085957NNH,1,29,1,202102
3,BRKR,20210228,A52013,사랑에 빠진 딸기,1038087873NNH,2,31,1,202102
4,BRKR,20210228,A52013,사랑에 빠진 딸기,1001761941NNH,2,42,1,202102
...,...,...,...,...,...,...,...,...,...
1061200,BRKR,20220201,A56087,아이스 로아커,1006237649YNH,2,50,1,202202
1061201,BRKR,20220201,A56087,아이스 로아커,1037770862NNH,1,55,1,202202
1061202,BRKR,20220201,A56087,아이스 로아커,1003279649YNH,2,37,1,202202
1061203,BRKR,20220201,A56087,아이스 로아커,1038624643NNH,2,57,1,202202


In [210]:
idx = 0
df[df.ITEM_NM==df.ITEM_NM.unique()[idx]]['ITEM_PLU_CD'].unique()

array(['A52013'], dtype=object)

In [205]:
# 이름으로 검색하는 함수 생성
nm = df.ITEM_NM.unique()
cd = df.ITEM_PLU_CD.unique()
item_d = {cd_c:nm_c for cd_c, nm_c in zip(cd,nm)}
def check_name(icecream):
    nm = df.ITEM_NM.unique()
    cd = df.ITEM_PLU_CD.unique()
    item_d = {cd_c:nm_c for cd_c, nm_c in zip(cd,nm)}
    df_item =pd.Series(item_d)
    check = df_item[df_item.str.contains(icecream)]
    return check,f'{len(check)}개'

check_name('초코')

(A52311           초코나무 숲
 A56040      아이스 촉촉한 초코칩
 A56050          다크초코나이트
 A56003    화이트 초코렛 마카다미안
 A56053    민트 초코렛칩 쿠키 도우
 A51889         민트 초코 봉봉
 A56071        체리 초코 쥬빌레
 A50108        초코 아몬드 봉봉
 dtype: object,
 '8개')

In [206]:
check_name('고디바')

(A56078    아이스 고디바 초콜릿
 dtype: object,
 '1개')

In [190]:
# 월별 판매량 집계
sale_df = df.pivot_table(values='SALE_QTY',aggfunc='sum',index='SALE_YM',columns=['ITEM_NM'], margins= True).T.fillna(0).sort_values('All',ascending=False)
sale_df.loc['아이스 고디바 초콜릿']

SALE_YM
202102      0.0
202103      0.0
202104      0.0
202105      0.0
202106      0.0
202107      0.0
202108      0.0
202109      0.0
202110      0.0
202111     42.0
202201    246.0
202202    117.0
All       405.0
Name: 아이스 고디바 초콜릿, dtype: float64

In [199]:
sale_df.loc[:,'202201'][:50]

ITEM_NM
All                1254109.0
엄마는 외계인             175442.0
민트 초콜릿 칩            110874.0
뉴욕치즈케이크              84211.0
레인보우 샤베트            103854.0
아몬드 봉봉               79624.0
체리쥬빌레                76178.0
사랑에 빠진 딸기            69334.0
31-요거트               68094.0
바람과함께사라지다            57020.0
베리베리 스트로베리           58094.0
그린티                  43640.0
슈팅스타                  8924.0
피스타치오 아몬드            35387.0
오레오 쿠키앤크림            32538.0
초콜릿 무스               17480.0
초콜릿                  24067.0
바닐라                  25595.0
초코나무 숲               42025.0
자모카 아몬드 훠지           13604.0
이상한 나라의 솜사탕           2424.0
쫀떡궁합                 16134.0
너는 참 달고나              2951.0
월넛                   11171.0
New 슈팅스타             17328.0
찰떡 콩떡                 4199.0
아이스 로아커                854.0
애플 민트                    4.0
NEW 이상한 나라의 솜사탕      11904.0
러브미                     40.0
오레오 쿠키 앤 민트           3085.0
오레오 쿠키 앤 크림치즈         5641.0
망고탱고                 10552.0
봉쥬르, 마카롱                16.0
워싱턴 블루

In [211]:
sale_df

SALE_YM,202102,202103,202104,202105,202106,202107,202108,202109,202110,202111,202201,202202,All
ITEM_NM,,,,,,,,,,,,,
All,1025242.0,1060396.0,1023186.0,1440210.0,1244293.0,1431159.0,1802169.0,1529697.0,1476669.0,1132520.0,1254109.0,1217584.0,15637234.0
엄마는 외계인,129834.0,143563.0,157999.0,188567.0,180458.0,216409.0,262205.0,217112.0,212209.0,169390.0,175442.0,150044.0,2203232.0
민트 초콜릿 칩,124173.0,127500.0,12755.0,154452.0,146900.0,148217.0,188202.0,154195.0,144059.0,108703.0,110874.0,95715.0,1515745.0
뉴욕치즈케이크,72264.0,72911.0,75619.0,91284.0,76484.0,102974.0,134131.0,106473.0,99958.0,72016.0,84211.0,75867.0,1064192.0
레인보우 샤베트,81192.0,80713.0,89375.0,112470.0,12510.0,11804.0,130573.0,114081.0,119865.0,97901.0,103854.0,97168.0,1051506.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
러브 포션 31,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
바람과 함께 사라지다_Cafe,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
민트 초코렛칩 쿠키 도우,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [185]:
# 월 평균 판매량 집계 함수 생성
def check_qty(item):
    month = sale_df.loc[item] != 0
    sale_mt_no = len(sale_df.loc[item][month])-1
    return round(sale_df.loc[item]['All']/sale_mt_no,2),sale_mt_no

In [186]:
check_qty('아이스 허니버터 아몬드')

(683.5, 12)

In [187]:
# 상품별 월평균판매량 정렬
item_d = {idx:[df.ITEM_PLU_CD.unique()[idx], nm, check_qty(nm)[0], check_qty(nm)[1]] for idx, nm in enumerate(df.ITEM_NM.unique())}
item_df = pd.DataFrame(item_d).T
columns=['ITEM_PLU_CD','ITEM_NM','SALE_QTY_MM','NO_SALE_MM']
item_df.columns = columns
item_df = item_df.fillna(0).sort_values('SALE_QTY_MM',ascending=False)

item_df                    

,ITEM_PLU_CD,ITEM_NM,SALE_QTY_MM,NO_SALE_MM
15,A51940,엄마는 외계인,183602.67,12
28,A50554,민트 초콜릿 칩,126312.08,12
27,A52198,뉴욕치즈케이크,88682.67,12
33,A56005,레인보우 샤베트,87625.50,12
37,A56038,아몬드 봉봉,80369.58,12
...,...,...,...,...
133,A56087,바람과 함께 사라지다_Cafe,1.00,1
134,A59993,프랄린앤크림,1.00,1
107,A56068,아이스 레모나,1.00,2
99,A52449,러브 포션 31,1.00,1


In [ ]:
item_df[:20].plot(kind='bar', x = 'ITEM_NM', y= 'SALE_QTY_MM', figsize = (15,5) , color = 'salmon', ylabel = '월평균판매량', title = '월평균 판매량 Top20 (21년2월 ~ 22년2월)')

In [ ]:
# 12개월 미만 판매제품 순위 (신상품 등)
item_df[item_df['NO_SALE_MM']<12]

In [112]:
item_df[item_df['NO_SALE_MM']<12][:20]['NO_SALE_MM']

,ITEM_PLU_CD,ITEM_NM,SALE_QTY_MM,NO_SALE_MM
132,A56086,아이스 로아커,25620.00,2
124,A56080,New 슈팅스타,14661.25,4
127,A56081,기운 센 아이스 콘푸로스트,10307.00,2
47,A52012,너는 참 달고나,9971.45,11
112,A56072,찰떡 콩떡,9548.17,6
120,A56079,NEW 이상한 나라의 솜사탕,7991.00,5
129,A56082,New 자모카 아몬드 훠지,7464.50,2
121,A56076,마법사의 할로윈,5869.67,3
89,A52197,애플 민트,4841.44,9
93,A59867,오레오 쿠키 앤 민트,4369.25,8


In [111]:
# 월, 나이대 범주화 테이블 생성
def age_range(age):
    import numpy as np
    if age <= 20 : return "~20"
    elif age <= 24 : return "20~24"
    elif age <= 29 : return "25~29"
    elif age <= 34 : return "30~34"
    elif age <= 39 : return "35~39"    
    elif age <= 44 : return "40~44"    
    elif age <= 49 : return "45~49"    
    elif age <= 54 : return "50~54"
    elif age <= 59 : return "55~59"
    elif age >= 60 : return "60~" 
    else : return np.NaN
df['PAY_AGE_RANGE'] = df['PAY_AGE'].apply(lambda x : age_range(x))
df.head(1)

# 월/그룹 별 테이블 생성
df_modi = df[['SALE_YM','ITEM_NM','ITEM_PLU_CD','PAY_GNDR_DIV','PAY_AGE_RANGE','SALE_QTY']]
function_list = ['sum']
group = df_modi.groupby(['SALE_YM','ITEM_NM','ITEM_PLU_CD','PAY_GNDR_DIV','PAY_AGE_RANGE'])
df_modi = df_modi.groupby(['SALE_YM','ITEM_NM','ITEM_PLU_CD','PAY_GNDR_DIV','PAY_AGE_RANGE'])['SALE_QTY'].agg(function_list).fillna(0).round(2).reset_index()
df_modi.columns = ['SALE_YM','ITEM_NM','ITEM_PLU_CD','PAY_GNDR_DIV','PAY_AGE_RANGE','SALE_QTY']
df_modi

,SALE_YM,ITEM_NM,ITEM_PLU_CD,PAY_GNDR_DIV,PAY_AGE_RANGE,SALE_QTY
0,202102,31-요거트,A55105,,~20,673
1,202102,31-요거트,A55105,1,20~24,387
2,202102,31-요거트,A55105,1,25~29,1112
3,202102,31-요거트,A55105,1,30~34,1389
4,202102,31-요거트,A55105,1,35~39,1559
...,...,...,...,...,...,...
13856,202202,화이트 초코렛 마카다미안,A51858,2,40~44,2
13857,202202,화이트 초코렛 마카다미안,A51858,2,45~49,2
13858,202202,화이트 초코렛 마카다미안,A51858,2,50~54,1
13859,202202,후르츠 환타지,A50891,1,45~49,2


In [115]:
pwd

'C:\\Users\\david\\Desktop\\Recommandation\\6_Recommandation'

In [116]:
item = pd.read_csv('./dataset/리스트.csv',encoding='euc-kr')
item

,Unnamed: 0,제품명,코드,컨셉만족도,외관만족도,맛만족도,전반만족도,구매의향,비고
0,'220427,오버더 레인보우 샤베트,A52197,5.1,5.5,5.4,5.4,5.0,NaN
1,'220427,꿀에 빠진 바나나와 망고,A56024,5.3,5.4,5.4,5.3,5.0,NaN
2,'220427,밀탑 팥빙수,NaN,5.0,4.1,5.0,5.1,4.9,NaN
3,'220427,던킨 에스프레소,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,'220511,꿀에 빠진 바나나와 망고,A56024,5.3,5.5,5.5,5.4,5.3,NaN
...,...,...,...,...,...,...,...,...,...
104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
105,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [130]:
columns = ['조사일자', '제품명', '코드', '컨셉만족도', '외관만족도', '맛만족도', '전반만족도', '구매의향', '비고']
item.columns = columns
item.제품명 = item.제품명.str.strip()
item.제품명

0       오버더 레인보우 샤베트
1      꿀에 빠진 바나나와 망고
2             밀탑 팥빙수
3           던킨 에스프레소
4      꿀에 빠진 바나나와 망고
           ...      
104              NaN
105              NaN
106              NaN
107              NaN
108              NaN
Name: 제품명, Length: 109, dtype: object

In [153]:
idx = 0
contents = ['ITEM_PLU_CD','SALE_QTY_MM','NO_SALE_MM']
a = item.iloc[[idx]]

In [151]:
b = item_df[item_df.ITEM_NM == item.제품명[idx]]

In [166]:
a

,조사일자,제품명,코드,컨셉만족도,외관만족도,맛만족도,전반만족도,구매의향,비고
0,'220427,오버더 레인보우 샤베트,A52197,5.1,5.5,5.4,5.4,5.0,NaN
